In [13]:
import csv
import pandas as pd
import statsmodels
import sqlite3
import sqlalchemy as sq
import numpy as np

In [14]:
fileList = ['2019-09-10.db','2019-09-17.db','2019-09-24.db']

In [48]:
def pullMarkets(marketName, filelist):
    keyFrame = []
    keyList = []
    #Retrieves the marketID from the SQL file
    for i in range(len(filelist)):
        dbName = (filelist[i])
        conn = sqlite3.connect(dbName)
        c = conn.cursor()
        c.execute("SELECT market_id FROM Markets WHERE market_name =?" , [marketName])
        df = c.fetchall()
        c.close()
        keyFrame.append(df)
    #Turns the Dataframe into a list. There is probably a better way to do this so that it is never a dataframe, but I don't
    #know it at the moment and this works
    for i in range(len(keyFrame)):
        keyList.append(keyFrame[i])
    return keyList

def numpy_sql_connection(conns):
    connections = [sqlite3.connect() for x in conns]
    return connections
    

def pullMarketsnumpy(marketName, filelist):
    filearray = np.asarray(filelist)
    sqlite_array = np.array(['sqlite:///']*conns.size)
    engine = create_engine(+conns[0:conns.size])
    keyFrame = []
    keyList = []
    
    #Retrieves the marketID from the SQL file
    for i in range(len(filelist)):
        dbName = (filelist[i])
        conn = sqlite3.connect(dbName)
        df = pd.read_sql("SELECT market_id FROM Markets WHERE market_name = '%s'" %marketName, conn)
        keyFrame.append(df)
    #Turns the Dataframe into a list. There is probably a better way to do this so that it is never a dataframe, but I don't
    #know it at the moment and this works
    for i in range(len(keyFrame)):
        keyList.append(keyFrame[i].iat[0,0])
    return keyList


#A sub function for the pullContract function, so that we can call "Pull Contract" as one function rather than a function and a loop.
#Have to do this to index both lists at the same time and have the ids match with the right database.
def subPullContract(marketID, contractName, filenum):
    keyFrame = []
    #Retrieves the contractID from the SQL file
    dbName = (filenum)
    conn = sqlite3.connect(dbName)
    df = pd.read_sql("SELECT contract_id FROM Contracts WHERE market_id = '%d' AND contract_name = '%s' " % (marketID, contractName), conn)
    keyFrame.append(df)
    #Turns the Dataframe into a list. There is probably a better way to do this so that it is never a dataframe, but I don't
    #know it at the moment and this works
    conVal = (keyFrame[0].iat[0,0])
    return conVal



def pullContract(marketIDs, contractName, filelist):
    contractIDs = []
    for i in range(len(filelist)):
        val = subPullContract(marketIDs[i], contractName, filelist[i])
        contractIDs.append(val)
    return contractIDs


def pullFullDataFrame(filelist, conNums):
    priceDF = pd.DataFrame()
    for i in range(len(filelist)):
        dbName = (filelist[i])
        conNum = (conNums[i])
        conn = sqlite3.connect(dbName)
        df = pd.read_sql("SELECT buy_yes FROM Prices WHERE contract_id = '%d' " % (conNum), conn)
        priceDF = df.append(priceDF, ignore_index = True)
   
        # Select from SQL command: Table: Prices
        #contractID: conNums[i]
        #value: buy_yes
        #send this to dataframe
        #append the info to a new dataframe
    return priceDF

def tradeRatio(x,y):
    z = x / y
    return z

In [49]:
print(pullMarketsbad('Who will win the 2020 Democratic presidential nomination?', fileList))

[[(19,)], [(19,)], [(19,)]]


In [50]:
marketNumDem = pullMarkets('Who will win the 2020 Democratic presidential nomination?', fileList)    
marketNumPrez = pullMarkets('Who will win the 2020 U.S. presidential election?', fileList)
    
joeConNom = pullContract(marketNumDem, 'Andrew Yang', fileList)
joeConPrez = pullContract(marketNumPrez, 'Andrew Yang', fileList)

bidenPrezPrice = pullFullDataFrame(fileList, joeConPrez)
bidenNominationPrice = pullFullDataFrame(fileList, joeConNom)

mergedset = pd.merge(bidenPrezPrice, bidenNominationPrice, left_index=True, right_index = True, how = 'outer')
mergedset['Ratio'] = mergedset[['buy_yes_x','buy_yes_y']].apply(lambda x: tradeRatio(x.buy_yes_x, x.buy_yes_y), axis=1)

print(mergedset)

mergedset.reset_index().plot(x='index', y = 'Ratio')


print('if you got nothing, failed')

TypeError: %d format: a number is required, not list